In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

import os
import sys
sys.path.append(os.path.join(
  os.path.abspath(''), '..', '..')
)

from src.preprocess import Preprocess

import warnings
warnings.filterwarnings('ignore')

In [ ]:
x_train: pd.DataFrame = pd.read_csv(
  '../../data/train.csv', index_col=0)
x_test: pd.DataFrame = pd.read_csv(
  '../../data/test.csv', index_col=0)
y_train: pd.Series = x_train['Survived']

x_train, x_test = Preprocess(scaler=StandardScaler()) \
  .apply(x_train, x_test)

params = {
  'n_estimators': [i for i in range(1, 16)],
}

base_boost = AdaBoostClassifier(random_state=3)
boost = GridSearchCV(
  base_boost,
  params,
  scoring='balanced_accuracy',
  n_jobs=-1,
  cv=5,
  return_train_score=True,
)
boost.fit(x_train, y_train)

results = pd.DataFrame(boost.cv_results_)
results = results[[
  'param_n_estimators','mean_test_score',
  'rank_test_score', 'mean_train_score'
]]
results.to_csv('boost_results.csv', index=False)

In [ ]:
plt.plot(results['param_n_estimators'],
  results['mean_test_score'], label='test')
plt.plot(results['param_n_estimators'],
  results['mean_train_score'], label='train')

plt.xlabel('Estimators')
plt.ylabel('Balanced Accuracy')
plt.title('Estimators Overfit Analysis')
plt.legend()
plt.savefig('estimators.svg', format='svg')